In [1]:
# Global imports
import os
import sys
import pathlib
import time

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn

from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.dataset import random_split
import torch.optim as optim
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler

In [2]:
cwd = pathlib.Path().resolve()
src = cwd.parent
root = src.parent
sys.path.append(str(src))

In [3]:
#initialize GPU -  In case of windows use cuda instead of nps
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("Is CUDA enabled?",torch.cuda.is_available())
print("Number of GPUs",torch.cuda.device_count())

Is CUDA enabled? False
Number of GPUs 0


###  Simple RNN 
Use the RNN model to predict the temporal evolution of flooding.\
The input time span is 10 simulation timesteps, and the output is 1 simulation timestep. 

In [4]:
from models.Networks import SimpleRNN
from utils.simulation import Simulation
from train import train_and_validate, evaluate_model
# from train import train_and_validate, evaluate_model

In [59]:
def create_sequences(series,T=10,H=1):
    # This function creates a dataset of input/output sequences from a time series.
    # The input sequence is T steps long, from time t to time t+T (excluded).
    # The output sequence is H steps long, from time t+T to time t+T+H (excluded).
    # Codes here represents that two type of dataset will be generated, the first one
    # (input time sequence) means that the dataset used for training, and the second one
    # (output time sequence) means that the dataset used for prediction, amd hence it is
    # distributed from span t+T to t+T+H
    X = []
    Y = []
    for t in range(len(series)-T-H):
        x = series[t:t+T]
        X.append(x)
        y = series[t+T:t+T+H]
        Y.append(y)
    X = np.array(X)
    Y = np.array(Y)
    return X,Y

In [60]:
sims = Simulation.load_simulations(str(root)+"/data/processed_data/normalized_training_data", sim_amount = 1, number_grids=64)
display(sims[0].wd.shape)

seqX, seqY = create_sequences(sims[0].wd, T =10, H =1)
display(seqX.shape)
display(seqY.shape)

(97, 64, 64)

(86, 10, 64, 64)

(86, 1, 64, 64)

In [61]:
# We keep track of indexes of train and validation.
seqX_tra, seqX_tst, seqY_tra, seqY_tst, iseqx_tra, iseqx_tst = train_test_split(
    seqX, seqY, np.arange(seqX.shape[0]), test_size=0.30, shuffle=True, random_state=42)

# Split the existing test dataset into validation and test sets (50/50 split)
seqX_val, seqX_tst, seqY_val, seqY_tst, iseqx_val, iseqx_tst = train_test_split(
    seqX_tst, seqY_tst, iseqx_tst, test_size=0.5, shuffle=True, random_state=42)


print(f"X_tra.shape: {seqX_tra.shape}, Y_tra.shape: {seqY_tra.shape}")
print(f"X_val.shape: {seqX_val.shape}, Y_val.shape: {seqY_val.shape}")
print(f"X_tst.shape: {seqX_tst.shape}, Y_tst.shape: {seqY_tst.shape}")

X_tra.shape: (60, 10, 64, 64), Y_tra.shape: (60, 1, 64, 64)
X_val.shape: (13, 10, 64, 64), Y_val.shape: (13, 1, 64, 64)
X_tst.shape: (13, 10, 64, 64), Y_tst.shape: (13, 1, 64, 64)


In [62]:
def train_and_validate(model, train_loader, val_loader, criterion, optimizer, num_epochs, device, save_path):
    best_val_loss = float("inf")  # Track the best validation loss
    train_losses = []
    val_losses = []

    start_time = time.time()  # Start training time

    for epoch in range(num_epochs):
        # Training Phase
        model.train()
        total_train_loss = 0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # Validation Phase
        avg_val_loss = evaluate_model(model, val_loader, criterion, device)
        val_losses.append(avg_val_loss)

        # Save Best Model
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), save_path)

        if (epoch + 1) % 20 == 0:
            print(f'Epoch {epoch+1}/{num_epochs}', f'Train Loss: {avg_train_loss:.4f}, '
                  f'Validation Loss: {avg_val_loss:.4f}', f'Best Validation Loss: {best_val_loss:.4f}')
    train_time = time.time() - start_time
    print("Training complete.")
    return train_losses, val_losses, best_val_loss, train_time

In [63]:
model_name = 'CNN-RNN'

batch_size=4
num_epochs = 200
lr = 0.0005
criterion = nn.MSELoss()
optimizer = optim.AdamW
model = SimpleRNN(10,1).to(device)

#create datasets and data loaders
train_dataset = TensorDataset(torch.tensor(seqX_tra, dtype=torch.float32), torch.tensor(seqY_tra, dtype=torch.float32))
val_dataset = TensorDataset(torch.tensor(seqX_val, dtype=torch.float32), torch.tensor(seqY_val, dtype=torch.float32))

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# defining the optimizer
optimizer = optimizer(model.parameters(), lr=lr)

# defining the save path
save_path = "../results/trained_models/" + model_name

# training
train_losses, val_losses, best_val_loss, time_RNN = train_and_validate(model, train_loader, val_loader, criterion, optimizer, num_epochs, device, save_path)

# Load the best model
model.load_state_dict(torch.load(save_path))

AssertionError: RNN: Expected input to be 2-D or 3-D but received 5-D tensor

In [ ]:
train_rnn(model = SimpleRNN,
          sim_amount=1,
          training_size=0.8,
          batch_size=4,
          num_epochs = 200,
          lr = 0.0005,
          criterion = nn.MSELoss(),
          optimizer = optim.AdamW,
          model_name = 'babies_first_RNN')

In [ ]:
simulation_length = 10 # number of simulation
grid_size = 64 # number of grid
timestep = 96 # number of time step

# Create a sample 2D array
input_data = torch.rand((sequence_length, input_size))

# Reshape into a 3D tensor (simulation_length, timestep, grid_size)
input_data = input_data.unsqueeze(1).expand(-1, batch_size, -1)

# Create an RNN layer
rnn = nn.RNN(input_size, hidden_size=10, num_layers=1, batch_first=True)


In [ ]:
display(input_data.shape)

In [ ]:
def loading(sim_amount):
    sims = Simulation.load_simulations(str(root)+"/data/processed_data/normalized_training_data", sim_amount = sim_amount, number_grids=64)

    n_timesteps = 96
    grid_size = 64
    channels = 2   # water depth and topography

    # reformat the data
    X = np.zeros((n_timesteps*len(sims), grid_size, grid_size, channels))   # timestep * grid_x * grid_y * channels
    Y = np.zeros(X[:,:,:,0].shape)   # timestep * grid_x * grid_y

    for i in range(len(sims)):   # number of simulations loaded in for training/validation

        sim = sims[i]            # get simulation
        topography = sim.topography

        for t_i in range(n_timesteps):    # number of timesteps

            wd, vx, vy = sim.return_timestep(t_i)
            X[t_i+i*n_timesteps, :, :, 0] = wd
            X[t_i+i*n_timesteps, :, :, 1] = topography

            wd, vx, vy = sim.return_timestep(t_i+1)
            Y[t_i+i*n_timesteps, :, :] = wd

    return(X,Y)

In [ ]:
X,Y = loading(sim_amount = 1)
display(X.shape) # input (DEM, WD) - one simulation
display(Y.shape) # output (WD) - one simulation